In [ ]:
# Clear all handlers first to start fresh
from loguru import logger
logger.remove()
import torch

from rag_app.services.rag import RagService

from llama_index.readers.file import PyMuPDFReader
from rag_app.services.ingest import IngestionService
from llama_index.core.node_parser import SemanticSplitterNodeParser

from rag_app.embeddings import get_chunk_embeddings, get_embed_model
from openai import AsyncOpenAI
from rag_app.config.settings import settings
from rag_app.core.vector_client import VectorClient
from rag_app.llm import get_llm_model
from sentence_transformers import CrossEncoder


c:\Users\mrudh\Documents\Projects\ProfileProject\rag_app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import nest_asyncio
nest_asyncio.apply()

reader = PyMuPDFReader()
chunk_embed_model = get_chunk_embeddings()
embedding_client = get_embed_model()

llm_model = get_llm_model()

vector_client = await VectorClient.create(
    api_key=settings.pinecone.api_key,
    environment=settings.pinecone.environment,
    index_name=settings.pinecone.index_name,
    dimension=settings.pinecone.dimension,
    metric=settings.pinecone.metric,
    cloud=settings.pinecone.cloud,
    region=settings.pinecone.region,
)

encoder_model = CrossEncoder(settings.reranker.model, device="cpu" if not torch.cuda.is_available() else "cuda")


node_parser = SemanticSplitterNodeParser(embed_model=chunk_embed_model)

ingest_service = IngestionService(
    reader=reader,
    node_parser=node_parser,
    llm_model=llm_model,
    vector_client=vector_client,
    embedding_client=embedding_client,
)

rag_service = RagService(
    embed_model=embedding_client,
    vector_client=vector_client,
    llm_model=llm_model,
    encoder_model=encoder_model)

2025-10-21 13:31:35 | INFO     | rag_app.embeddings:get_chunk_embeddings:23 - Using llama-index Local Embedding Model
2025-10-21 13:31:35 | INFO     | rag_app.embeddings:get_embed_model:39 - Initializing Local Models Embedding Model
2025-10-21 13:31:35 | INFO     | rag_app.llm:get_llm_model:16 - Initializing Local LLM


In [3]:
# nodes = await ingest_service.ingest(file_path="C:/Users/mrudh/Documents/Data/CWMG-KS-Vol-001-I.pdf")

In [4]:
query = "What was Gandhi’s early argument in favor of vegetarianism?"
matches = await rag_service.answer_query(query=query)

2025-10-21 13:31:42 | INFO     | rag_app.services.rag:answer_query:159 - Starting RAG retrieval for query: What was Gandhi’s early argument in favor of vegetarianism?
2025-10-21 13:31:47 | INFO     | rag_app.services.rag:answer_query:161 - Rewritten query: Can you provide details on the early motivations and arguments that led Mahatma Gandhi to adopt a vegetarian lifestyle?
2025-10-21 13:31:47 | INFO     | rag_app.services.rag:answer_query:163 - Generating embedding for rewritten query.
2025-10-21 13:31:47 | INFO     | rag_app.services.rag:answer_query:166 - Retrieving documents and questions from vector store.
2025-10-21 13:31:50 | INFO     | rag_app.services.rag:answer_query:169 - Retrieved 10 relevant documents and 10 relevant questions.
2025-10-21 13:31:50 | INFO     | rag_app.services.rag:answer_query:177 - Reranking the retrieved results.
2025-10-21 13:31:53 | INFO     | rag_app.services.rag:answer_query:180 - Generating final answer from top contexts.
2025-10-21 13:32:28 | INFO 

In [5]:
print(matches['answer'])

Based on the provided contexts, here’s a breakdown of the early motivations and arguments behind Gandhi’s adoption of vegetarianism:

*   **Confession and Ahimsa:** A pivotal moment was a childhood confession involving stealing gold from his brother. This led to a profound realization about the power of *ahimsa* (non-violence) and a desire to avoid causing harm to others, influencing his dietary choices. (Source: 36, 42)
*   **Indian Vegetarianism:** Gandhi became increasingly concerned about the health and well-being of Indians, particularly in the context of British rule and the effects of colonial diets. He saw vegetarianism as a way to promote physical and moral strength among Indians. (Source: 39, 41, 47)
*   **Influence of the Vegetarian Movement:** Gandhi was deeply involved in the Indian Vegetarian Movement, advocating for its spread and promoting its benefits. (Source: 32, 33, 35, 38)
*   **Experiment in Vital Food:** He was interested in exploring "vital food" and believed ve

In [6]:
matches['sources']

[{'content': '[ xxx ] \n30 \nWORK FOR VEGETARIANISM (30-9-1893) \n65 \n31 \nGUIDE TO LONDON (1893-94) \n66 \n32 \nAN EXPERIMENT IN VITAL FOOD (24-3-1894) \n121 \n33 \nLETTER TO "THE VEGETARIAN" (28-4-1894) \n125 \n34 \nVEGETARIANISM AND CHILDREN (5-5-1894) \n126 \n35 \nQUESTIONS ON RELIGION (Before June 1894) \n127 \n36 \nPETITION TO NATAL LEGISLATIVE ASSEMBLY (28-6-1894) \n128 \n37 \nDEPUTATION TO NATAL PREMIER (29-6-1894) \n133 \n38 \nA CIRCULAR LETTER TO LEGISLATORS (1-7-1894) \n135 \n39 \nDEPUTATION TO NATAL GOVERNOR (3-7-1894) \n136 \n40 \nPETITION TO NATAL LEGISLATIVE COUNCIL (4-7-1894) \n138 \n41 \nEXTRACT FROM LETTER TO DADABHAI NAOROJI (5-7-1894) \n139 \n42 \nPETITION TO NATAL LEGISLATIVE COUNCIL (6-7-1894) \n140 \n43 \nLETTER TO "THE NATAL MERCURY" (7-7-1894) \n144 \n44 \nPETITION TO NATAL GOVERNOR (10-7-1894) \n146 \n45 \nPETITION TO LORD RIPON (Before 14-7-1894) \n147 \n46 \nLETTER TO DADABHAI NAOROJI (14-7-1894) \n157 \n47 \nLETTER TO DADABHAI NAOROJI (27-7-1894) \n158 \n4

In [2]:
from google import genai

client = genai.Client()

result = client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

[ContentEmbedding(
  values=[
    -0.022374554,
    -0.004560777,
    0.013309286,
    -0.0545072,
    -0.02090443,
    <... 3067 more items ...>,
  ]
)]


[-0.022374554,
 -0.004560777,
 0.013309286,
 -0.0545072,
 -0.02090443,
 0.012355714,
 0.015772128,
 0.0054723006,
 0.031729158,
 0.0058553913,
 0.027073925,
 -0.0045324513,
 -0.01544016,
 0.031618375,
 0.121548004,
 0.01925409,
 0.0008599909,
 0.0061733276,
 -0.009662611,
 -0.015545654,
 0.017062597,
 -0.008637558,
 -0.017125048,
 0.0077396077,
 -0.0153139075,
 0.011430326,
 0.020329107,
 -0.00451,
 0.024133444,
 0.0070407446,
 0.020197608,
 0.0015623937,
 -0.008911156,
 0.028138846,
 -0.017435355,
 -0.012656962,
 0.009481721,
 -0.016410641,
 -0.015019126,
 0.0144167375,
 -0.023614116,
 -0.010397569,
 -0.0024164703,
 -0.019404082,
 0.019276941,
 -0.011112846,
 0.014422737,
 -0.042639293,
 -0.014391114,
 0.008184219,
 -0.012202394,
 0.012318022,
 -0.010061107,
 -0.15826157,
 0.015428178,
 0.01053043,
 -0.0069031497,
 -0.010121202,
 -0.025695775,
 -0.028000489,
 -0.0070861652,
 -0.014084083,
 -0.008487853,
 -0.022122845,
 0.0084621115,
 -0.008990799,
 -0.020783192,
 0.010838266,
 0.00139